# DQN vs Double DQN - Implementación C++ con CUDA

Este notebook ejecuta la comparación de DQN y Double DQN usando C++ con aceleración CUDA en Google Colab.

## Requisitos
- Runtime con GPU (Menú: Runtime > Change runtime type > GPU)

In [ ]:
# Verificar GPU disponible
!nvidia-smi

In [ ]:
# Clonar repositorio
!git clone https://github.com/ChristianPE1/Labs-Robotica-EPCC.git
%cd Labs-Robotica-EPCC/lab-6-ddqn-cpp

In [ ]:
# Ver archivos
!ls -la

In [ ]:
# Compilar versión CUDA
!nvcc -std=c++17 -O3 -arch=sm_75 -o train_cuda main_cuda.cu

In [ ]:
# Ejecutar entrenamiento con 5000 episodios (aprox 5-10 min)
!./train_cuda 5000

In [ ]:
# Visualizar resultados
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Cargar datos
dqn_df = pd.read_csv('dqn_metrics.csv')
ddqn_df = pd.read_csv('ddqn_metrics.csv')

print(f"DQN - Episodios: {len(dqn_df)}, Reward promedio: {dqn_df['reward'].mean():.2f}")
print(f"DDQN - Episodios: {len(ddqn_df)}, Reward promedio: {ddqn_df['reward'].mean():.2f}")

In [ ]:
# Función para calcular media móvil
def moving_average(data, window=100):
    return pd.Series(data).rolling(window=window, min_periods=1).mean()

# Crear figura con subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Recompensas
ax1 = axes[0, 0]
ax1.plot(dqn_df['episode'], moving_average(dqn_df['reward']), label='DQN', alpha=0.8)
ax1.plot(ddqn_df['episode'], moving_average(ddqn_df['reward']), label='Double DQN', alpha=0.8)
ax1.axhline(y=500, color='green', linestyle='--', alpha=0.5, label='Meta (500)')
ax1.set_xlabel('Episodio')
ax1.set_ylabel('Recompensa (media móvil 100)')
ax1.set_title('Recompensas por Episodio')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Tasa de éxito acumulada
ax2 = axes[0, 1]
dqn_success = (dqn_df['length'] >= 500).cumsum() / (dqn_df['episode']) * 100
ddqn_success = (ddqn_df['length'] >= 500).cumsum() / (ddqn_df['episode']) * 100
ax2.plot(dqn_df['episode'], dqn_success, label='DQN', alpha=0.8)
ax2.plot(ddqn_df['episode'], ddqn_success, label='Double DQN', alpha=0.8)
ax2.set_xlabel('Episodio')
ax2.set_ylabel('Tasa de Éxito (%)')
ax2.set_title('Tasa de Éxito Acumulada')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Pérdida
ax3 = axes[1, 0]
ax3.plot(dqn_df['episode'], moving_average(dqn_df['loss']), label='DQN', alpha=0.8)
ax3.plot(ddqn_df['episode'], moving_average(ddqn_df['loss']), label='Double DQN', alpha=0.8)
ax3.set_xlabel('Episodio')
ax3.set_ylabel('Pérdida (media móvil 100)')
ax3.set_title('Pérdida durante Entrenamiento')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Epsilon
ax4 = axes[1, 1]
ax4.plot(dqn_df['episode'], dqn_df['epsilon'], label='DQN', alpha=0.8)
ax4.plot(ddqn_df['episode'], ddqn_df['epsilon'], label='Double DQN', alpha=0.8)
ax4.set_xlabel('Episodio')
ax4.set_ylabel('Epsilon')
ax4.set_title('Decaimiento de Epsilon')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('comparison_cuda.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nGráfico guardado como 'comparison_cuda.png'")

In [ ]:
# Resumen estadístico
print("\n" + "="*60)
print("RESUMEN ESTADÍSTICO")
print("="*60)

# Últimos 500 episodios para métricas más estables
last_n = 500
dqn_last = dqn_df.tail(last_n)
ddqn_last = ddqn_df.tail(last_n)

print(f"\n{'Métrica':<30} {'DQN':>15} {'Double DQN':>15}")
print("-"*60)
print(f"{'Reward promedio (total)':<30} {dqn_df['reward'].mean():>15.2f} {ddqn_df['reward'].mean():>15.2f}")
print(f"{'Reward promedio (últimos 500)':<30} {dqn_last['reward'].mean():>15.2f} {ddqn_last['reward'].mean():>15.2f}")
print(f"{'Reward máximo':<30} {dqn_df['reward'].max():>15.0f} {ddqn_df['reward'].max():>15.0f}")
print(f"{'Desviación estándar':<30} {dqn_df['reward'].std():>15.2f} {ddqn_df['reward'].std():>15.2f}")
print(f"{'Tasa de éxito total (%)':<30} {(dqn_df["length"] >= 500).sum() / len(dqn_df) * 100:>15.2f} {(ddqn_df["length"] >= 500).sum() / len(ddqn_df) * 100:>15.2f}")
print(f"{'Episodios exitosos':<30} {(dqn_df['length'] >= 500).sum():>15} {(ddqn_df['length'] >= 500).sum():>15}")

In [ ]:
# Descargar resultados
from google.colab import files

files.download('dqn_metrics.csv')
files.download('ddqn_metrics.csv')
files.download('comparison_cuda.png')